In [ ]:
pip install arxiv wikipedia langchain_community langchain_openai langchain_text_splitters

In [ ]:
pip install faiss-cpu langchain_groq

In [3]:
#@title Creating Tools
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
## using the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max= 250)

#run wrapper
wiki = WikipediaQueryRun(api_wrapper= api_wrapper_wiki)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [5]:
wiki.name

'wikipedia'

In [6]:
## using the inbuilt tool of arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max= 250)

#run wrapper
arxiv = ArxivQueryRun(api_wrapper= api_wrapper_arxiv)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [7]:
arxiv.name

'arxiv'

In [8]:
# combining both wikipedia + arxiv

tools= [wiki,arxiv]

In [9]:
import os

OPENAI_API_KEY= "**********************"
LANGCHAIN_API_KEY = "**********************"
GROQ_API_KEY= "**********************"
HF_TOKEN = "**********************"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LANGCHAIN_PROJECT"
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [10]:
#@title Create custom tool [RAG tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [11]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)

retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b43e7a93850>, search_kwargs={})

In [12]:
#@title to use retriver as a tool , we have to convert it into a "tool"

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, name="langsmith-search", description="Search any information about Langsmith.")
retriever_tool

Tool(name='langsmith-search', description='Search any information about Langsmith.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7b43e5b8aac0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b43e7a93850>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7b43e5ce4400>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b43e7a93850>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', 

In [13]:
print(retriever_tool.name)
print(retriever_tool.description)

langsmith-search
Search any information about Langsmith.


In [14]:
#@title combining tools

tools = [wiki, arxiv, retriever_tool]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7b43e5b8aac0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b43e7a93850>, search_kwargs={}), document

In [18]:
#@title Run the tools using with "agents" and "llm models"
from langchain_groq import ChatGroq
import openai

llm = ChatGroq(model="Llama3-8b-8192")

In [20]:
#@title Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")  #sample prompt for agents
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
#@title creating "agent"
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [26]:
#@title running the "agent"

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [27]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic ‚Äî¬†it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability‚Äã
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith‚Äôs observability features have you covered throughout all stages of application develo

{'input': 'Tell me about Langsmith',
 'output': 'It seems like Langsmith is a platform for building production-grade Large Language Model (LLM) applications. It provides features for observability, evaluation, and prompt engineering to help developers monitor, evaluate, and optimize their LLM applications. Langsmith allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. It also provides tools for iterating on prompts, with automatic version control and collaboration features.\n\nWith Langsmith, you can analyze traces in the platform and configure metrics, dashboards, and alerts based on these. You can also evaluate your application over production traffic, score application performance, and get human feedback on your data. Additionally, Langsmith provides off-the-shelf evaluators to help you quickly assess the performance of your application.\n\nIt seems like Langsmith is a comprehensive platform for building and deploying LLM applicat

In [28]:
agent_executor.invoke({"input":"what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the yielded results, I can provide a direct response.

Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and improve their performance on a task over time. The goal of machine learning is to enable machines to make predictions, classify data, or make decisions without being explicitly programmed to do so. Machine learning algorithms can learn from both labeled and unlabeled data and can be used to solve a wide range of problems, including image and speech recognition, natural language processing, and predictive modeling.

> Fini

{'input': 'what is machine learning',
 'output': 'Based on the yielded results, I can provide a direct response.\n\nMachine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and improve their performance on a task over time. The goal of machine learning is to enable machines to make predictions, classify data, or make decisions without being explicitly programmed to do so. Machine learning algorithms can learn from both labeled and unlabeled data and can be used to solve a wide range of problems, including image and speech recognition, natural language processing, and predictive modeling.'}

In [29]:
agent_executor.invoke({"input":" what is this research paper 1706.03762 about"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt looks like the tool call yielded a result! Based on the result, it appears that the research paper 1706.03762 is titled "Attention Is All You Need" and was published on August 2, 2023.

> Finished chain.


{'input': ' what is this research paper 1706.03762 about',
 'output': 'It looks like the tool call yielded a result! Based on the result, it appears that the research paper 1706.03762 is titled "Attention Is All You Need" and was published on August 2, 2023.'}